# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "cities.csv"

df = pd.read_csv(file)
df.head()


,Unnamed: 0,City,Temp (F),Humidity,Cloudiness,Wind Speed (MPH),Latitude,Longitude
0,0,Busselton,92.25,9,4,5.68,-33.65,115.33
1,1,Saint-Joseph,78.80,54,0,23.04,-21.37,55.62
2,2,Balkanabat,26.92,80,0,0.96,39.51,54.37
3,3,Punta Arenas,44.29,81,0,5.82,-53.15,-70.92
4,4,Rikitea,77.22,79,60,14.23,-23.12,-134.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = list(zip(df['Latitude'],df['Longitude']))
humidity = df['Humidity'].astype('float')

gmaps.configure(api_key=g_key)

fig = gmaps.figure(center=(46.0, -5.0),zoom_level = 1)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
hotel_df = df.loc[(df['Temp (F)'] < 45) & (df['Temp (F)'] > 20) & (df['Wind Speed (MPH)'] > 12) & (df['Cloudiness'] < 5)]
hotel_df = hotel_df.dropna()
hotel_df.count()

Unnamed: 0          5
City                5
Temp (F)            5
Humidity            5
Cloudiness          5
Wind Speed (MPH)    5
Latitude            5
Longitude           5
dtype: int64

In [20]:
hotel_df = hotel_df.drop(columns= ['Unnamed: 0'])
hotel_df['Hotel Name'] = " "
hotel_df['Country'] = " "

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# NOTE: Do not change any of the code in this cell
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
placeid = []
locations = list(zip(hotel_df['Latitude'],hotel_df['Longitude']))

for place in locations:
    lat = place[0]
    lon = place[1]
    coord = str(lat) + ", " + str(lon)
    params = {
        'location':coord,
        'radius':5000,
        'type':'hotel',
        'key':g_key
        }
    try:
        response = requests.get(url,params=params).json()
        hotelname = response['results'][1]['name']
        placeid.append(response['results'][1]['place_id'])
        hotel_df.loc[(hotel_df['Latitude'] == lat) & (hotel_df['Longitude'] == lon), 'Hotel Name'] = hotelname
    except:
        print ("Can't find a hotel near " + coord)



In [25]:
url2 = "https://maps.googleapis.com/maps/api/place/details/json"
countries = []
for pid in placeid:
    params = {
        'placeid':pid,
        'key':g_key
        }
    try:
        response = requests.get(url2,params=params).json()
        try:
            x = int(response['result']['address_components'][5]['long_name'])
            countries.append(response['result']['address_components'][4]['long_name'])
        except:
            countries.append(response['result']['address_components'][5]['long_name'])
    except:
        try:
            response = requests.get(url2,params=params).json()
            countries.append(response['result']['address_components'][3]['long_name'])
        except:
            print('New?')

In [26]:
hotel_df['Country'] = countries
hotel_df

,City,Temp (F),Humidity,Cloudiness,Wind Speed (MPH),Latitude,Longitude,Hotel Name,Country
186,Novyy Svit,25.27,93,0,16.24,47.81,38.02,Novosvit·sʹka Zahalʹnoosvitnya Shkola I-Iii St...,Ukraine
227,Zenzeli,24.33,94,0,13.35,45.92,47.05,Bankomat Pao Sberbank,Russia
274,South Sioux City,36.12,74,1,20.80,42.47,-96.41,Hilton Garden Inn Sioux City Riverfront,United States
551,Beatrice,40.14,65,1,12.75,40.27,-96.75,Victorian Inn Beatrice,United States
564,Almaznyy,23.00,79,0,17.90,48.04,40.05,Krasnosulinskaya Ptitsefabrika,Russia


In [27]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Show fig
fig




Figure(layout=FigureLayout(height='420px'))